# Tutorial bookkeeping

In this notebook, we explain how to use the LaCE bookkeeping

## Read data

In [ ]:
from lace.emulator import pd_archive

#### Read 768 post-processing with central simulation

In [2]:
archive = pd_archive.archivePD()
len(archive.data)

10230

In [4]:
# nsim * nphases * naxes * nscalings * nsnaps
31 * 2 * 3 * 5 * 11

10230

#### Read 768 post-processing without central simulation

In [5]:
archive = pd_archive.archivePD(drop_sim="central")
len(archive.data)

9900

In [6]:
# nsim * nphases * naxes * nscalings * nsnaps
30 * 2 * 3 * 5 * 11

9900

#### Read 768 post-processing dropping random simulation (but including central)

In [7]:
archive = pd_archive.archivePD(drop_sim=10)
len(archive.data)

9900

In [8]:
# nsim * nphases * naxes * nscalings * nsnaps
30 * 2 * 3 * 5 * 11

9900

#### Read h simulation from 768 post-processing 
Also works for nu, central, or a number

In [14]:
archive = pd_archive.archivePD(pick_sim="h")
len(archive.data)

330

In [15]:
# nsim * nphases * naxes * nscalings * nsnaps
1 * 2 * 3 * 5 * 11

330

#### Read 500 post-processing without central simulation

In [16]:
archive = pd_archive.archivePD(post_processing="500")
len(archive.data)

1980

In [17]:
# nsim * nphases * naxes * nscalings * nsnaps
30 * 2 * 1 * 3 * 11

1980

#### Read h simulation from 500 post-processing 
Also works for nu, central, or a number

In [19]:
archive = pd_archive.archivePD(post_processing="500", pick_sim="h")
len(archive.data)

22

In [21]:
# nsim * nphases * naxes * nscalings * nsnaps
1 * 2 * 1 * 1 * 11

22

## Compute averages

#### Read 768 post-processing with central simulation, then compute different types of averages
It also works for other options

In [22]:
archive = pd_archive.archivePD()
len(archive.data)

10230

In [25]:
# nsim * nphases * naxes * nscalings * nsnaps
31 * 2 * 3 * 5 * 11

10230

In [26]:
archive.average_over_samples(flag="all")
archive.average_over_samples(flag="phases")
archive.average_over_samples(flag="axes")
print(
    len(archive.data_av_phases),
    len(archive.data_av_axes),
    len(archive.data_av_all),
)

5115 3410 1705


In [27]:
# nsim * nphases * naxes * nscalings * nsnaps
print(31 * 1 * 3 * 5 * 11)
print(31 * 2 * 1 * 5 * 11)
print(31 * 1 * 1 * 5 * 11)

5115
3410
1705


## Inputs for emulator

#### Read 768 post-processing with central simulation, then compute different types of averages, finally produce different types of input for emulator
It also works for other options

In [22]:
archive = pd_archive.archivePD()
len(archive.data)

10230

In [25]:
# nsim * nphases * naxes * nscalings * nsnaps
31 * 2 * 3 * 5 * 11

10230

In [26]:
archive.average_over_samples(flag="all")
archive.average_over_samples(flag="phases")
archive.average_over_samples(flag="axes")
print(
    len(archive.data_av_phases),
    len(archive.data_av_axes),
    len(archive.data_av_all),
)

5115 3410 1705


In [27]:
# nsim * nphases * naxes * nscalings * nsnaps
print(31 * 1 * 3 * 5 * 11)
print(31 * 2 * 1 * 5 * 11)
print(31 * 1 * 1 * 5 * 11)

5115
3410
1705


In [28]:
archive.input_emulator(flag="all")
archive.input_emulator(flag="phases")
archive.input_emulator(flag="axes")
print(
    len(archive.data_input_all),
    len(archive.data_input_phases),
    len(archive.data_input_axes),
)

11935 6820 5115


In [35]:
# nsim * nphases * naxes * nscalings * nsnaps
# invidual + average all
print(31 * 2 * 3 * 5 * 11 + 31 * 1 * 1 * 5 * 11)
# average phases + average all
print(31 * 1 * 3 * 5 * 11 + 31 * 1 * 1 * 5 * 11)
# average axes + average all
print(31 * 2 * 1 * 5 * 11 + 31 * 1 * 1 * 5 * 11)

11935
6820
5115
